In [1]:
import numpy as np
import pandas as pd
import os
from PIL import Image
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from keras.utils import to_categorical

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, random_split, DataLoader
import torchvision.transforms as T
from torchvision import models

In [2]:
lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'dermatofibroma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}
lesion_categorical = {k:i for i, k in enumerate(lesion_type_dict)}

In [3]:
DATA_DIR = '../input/skin-cancer-mnist-ham10000'
original_df = pd.read_csv(os.path.join(DATA_DIR, 'HAM10000_metadata.csv'))

In [4]:
original_df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [5]:
original_df['localization'].unique()

array(['scalp', 'ear', 'face', 'back', 'trunk', 'chest',
       'upper extremity', 'abdomen', 'unknown', 'lower extremity',
       'genital', 'neck', 'hand', 'foot', 'acral'], dtype=object)

In [6]:
localization_to_index = {name:i for i, name in enumerate(original_df['localization'].unique())}

In [7]:
original_df['lesion_id'].nunique()

7470

In [8]:
# Drop dx_type since we don't care about it when predicting dx
temp_df = original_df.drop('dx_type', axis = 1)
# Convert dx to categorical
temp_df['dx'] = temp_df['dx'].map(lesion_categorical)
# Drop null values
temp_df = temp_df.dropna()
temp_df.head()

,lesion_id,image_id,dx,age,sex,localization
0,HAM_0000118,ISIC_0027419,2,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,2,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,2,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,2,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,2,75.0,male,ear


## Perform upsampling
There is a huge class imbalance in the dataset. Let's first demonstrate it and then attempt to solve it by upsampling.

In [9]:
freqs = temp_df['dx'].value_counts()
freqs

0    6660
1    1111
2    1089
3     514
4     327
5     142
6     115
Name: dx, dtype: int64

The disease with index 0 has imbalanced the other diseases. Let's perform upsampling to solve this issue.

In [10]:
upsample_rate = [6705 // freq for freq in freqs]

for i in range(1, 7):
    temp_df = temp_df.append([temp_df.loc[temp_df['dx'] == i,:]]*(upsample_rate[i]-1), ignore_index=True)
    
temp_df['dx'].value_counts(normalize=True)

3    0.143928
5    0.143756
6    0.143669
1    0.143583
0    0.143454
4    0.140869
2    0.140740
Name: dx, dtype: float64

# Note
**This step is optional. I found that only class 0 has duplicates. So if you perform it, your validation set will only contain one class, which is way worse than the problem we're trying to avoid. Think of a way to solve it and tell me; thanks!**

## Creating Train and Validation Sets
We know that there are duplicate lesion id's in the dataset. This is because each lesion id maps to more than 1 image id. So, if we split right away, duplicates might end up in the train and validation set. To avoid this, perform the following steps:
1. Create a column that indicates whether a lesion id has duplicates or not
2. Create training set with rows that have duplicates
3. Perform a random split on rows without duplicates
4. The result of the split will be the validation set and another training set that will be concatenated to the original


In [11]:
#duplicates = temp_df.groupby('lesion_id')['image_id'].count()
#duplicates.head()
#temp_df['has_duplicates'] = temp_df['lesion_id'].apply(
#    lambda lesion_id: duplicates[lesion_id] > 1
#)
#temp_df['has_duplicates'].value_counts()
#train_df = temp_df[temp_df['has_duplicates']]
#mixed_df = temp_df[~temp_df['has_duplicates']]
#train_df['dx'].value_counts(normalize=True)
#val_df['dx'].value_counts(normalize=True)
#mixed_df.shape[0], train_df.shape[0]
temp_df.shape[0]

46426

## Decide lengths of train and val dataframes

In [12]:
train_df, val_df = train_test_split(temp_df, test_size=0.2)
train_df.shape[0], val_df.shape[0]

(37140, 9286)

In [13]:
#train_df = pd.concat([train_df, train_temp], ignore_index=True)
#train_df.shape[0], val_df.shape[0]

In [14]:
# Drop the has_duplicates column since we don't need it anymore
#train_df = train_df.drop('has_duplicates', axis = 1)
#val_df = val_df.drop('has_duplicates', axis = 1).reset_index()

In [15]:
train_df.reset_index(inplace=True, drop=True)
val_df.reset_index(inplace=True, drop=True)

In [16]:
train_df.head()

,lesion_id,image_id,dx,age,sex,localization
0,HAM_0003110,ISIC_0032114,6,65.0,male,lower extremity
1,HAM_0000005,ISIC_0024579,4,75.0,female,lower extremity
2,HAM_0004423,ISIC_0026737,2,65.0,female,upper extremity
3,HAM_0002243,ISIC_0033019,3,80.0,male,back
4,HAM_0006371,ISIC_0033780,6,35.0,female,lower extremity


In [17]:
val_df.head()

,lesion_id,image_id,dx,age,sex,localization
0,HAM_0007089,ISIC_0030066,0,35.0,male,chest
1,HAM_0005346,ISIC_0028171,0,75.0,male,lower extremity
2,HAM_0004881,ISIC_0032913,1,85.0,male,abdomen
3,HAM_0007051,ISIC_0031002,6,65.0,male,upper extremity
4,HAM_0002787,ISIC_0025425,5,50.0,male,trunk


# Create Dataset

In [18]:
# https://www.kaggle.com/xinruizhuang/skin-lesion-classification-acc-90-pytorch
mean = [0.763038, 0.54564667, 0.57004464]
std = [0.14092727, 0.15261286, 0.1699712]

In [19]:
class SkinCancerDataset(Dataset):
    def __init__(self, df, transforms = None):
        self.df = df
        self.transforms = transforms
        self.part1 = os.path.join(DATA_DIR, 'HAM10000_images_part_1')
        self.part2 = os.path.join(DATA_DIR, 'HAM10000_images_part_2')
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.loc[idx]
        image_id = row['image_id']
        dx = row['dx']
        features = list(row.loc['age':'localization'])
        age, sex, localization = features[0], features[1], features[2]
        sex = 0 if sex == 'male' else 1
        localization = localization_to_index[localization]
        sex = to_categorical(sex, num_classes = 2)
        localization = to_categorical(localization, num_classes = len(localization_to_index))
        
        try:
            img = Image.open(os.path.join(self.part1, image_id + '.jpg'))
        except FileNotFoundError:
            img = Image.open(os.path.join(self.part2, image_id + '.jpg'))
            
        if self.transforms != None:
            img = self.transforms(img)

        return  torch.tensor([age], dtype=torch.float32), torch.tensor(sex, dtype=torch.float32),\
                torch.tensor(localization, dtype=torch.float32), img, dx

In [20]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [21]:
device = get_default_device()
device

device(type='cuda')

# Modeling with Pretrained Networks

In [22]:
# feature_extract is a boolean that defines if we are finetuning or feature extracting. 
# If feature_extract = False, the model is finetuned and all model parameters are updated. 
# If feature_extract = True, only the last layer parameters are updated, the others remain fixed.
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [23]:
def initialize_model(model_name, num_classes, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    
    assert model_name in ['resnet', 'vgg', 'densenet', 'inception'], 'Invalid model name'

    if model_name == "resnet":
        """ Resnet18, resnet34, resnet50, resnet101 """
        model_ft = models.resnet50(pretrained=use_pretrained)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224


    elif model_name == "vgg":
        """ VGG11_bn """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224


    elif model_name == "densenet":
        """ Densenet121 """
        model_ft = models.densenet121(pretrained=use_pretrained)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    else:
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    return model_ft, input_size

In [24]:
model_name = 'densenet'
model_ft, image_size = initialize_model(model_name, 20, use_pretrained=True)

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


In [25]:
class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        age, sex, localization, image, label = batch
        out = self(image, age, sex, localization) 
        loss = F.cross_entropy(out, label)      
        return loss
    
    def validation_step(self, batch):
        age, sex, localization, image, label = batch
        out = self(image, age, sex, localization)
        loss = F.cross_entropy(out, label)
        score = F_score(out, label)
        return {'val_loss': loss.detach(), 'val_score': score.detach() }
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_scores = [x['val_score'] for x in outputs]
        epoch_score = torch.stack(batch_scores).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_score': epoch_score.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], last_lr: {:.4f}, train_loss: {:.4f}, val_loss: {:.4f}, val_score: {:.4f}".format(
            epoch, result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_score']))


In [26]:
class MyModel(ImageClassificationBase):
    def __init__(self, model_name):
        super().__init__()
        self.cnn = model_ft
        self.fc1 = nn.Linear(20 + 1 + 2 + 15, 64)
        self.fc2 = nn.Linear(64, 16)
        self.fc3 = nn.Linear(16, 7)
        
    def forward(self, image, age, sex, localization):
        x1 = self.cnn(image)
        x = torch.cat([x1, age, sex, localization], dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [27]:
train_ds = SkinCancerDataset(train_df, transforms=T.Compose([
                                                                T.RandomCrop(image_size, padding=8, padding_mode='reflect'),
                                                                T.ToTensor(),
                                                                T.Normalize(mean, std, inplace=True)
                                                            ])
                            )
val_ds = SkinCancerDataset(val_df, transforms=T.Compose([
                                                                T.RandomCrop(image_size, padding=8, padding_mode='reflect'),
                                                                T.ToTensor(),
                                                                T.Normalize(mean, std, inplace=True)
                                                            ])
                            )

In [28]:
batch_size = 32

train_dl = DataLoader(train_ds, batch_size, shuffle=True, 
                      num_workers=3, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size*2, 
                    num_workers=2, pin_memory=True)

train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)

In [29]:
model = MyModel(model_name)
model = model.to(device)

In [30]:
batch_size = 1
image = torch.randn(batch_size, 3, 224, 224).to(device)
age = torch.randn(batch_size, 1).to(device)
sex = torch.randn(batch_size, 2).to(device)
localization = torch.randn(batch_size, 15).to(device)

In [31]:
model(image, age, sex, localization)

tensor([[ 0.0624, -0.2529, -0.2883, -0.0562, -0.2014,  0.0304,  0.4079]],
       device='cuda:0', grad_fn=<AddmmBackward>)

# Training Phase

In [32]:
def F_score(output, label, beta=1):
    _, prob = output.max(dim=1)
    TP = (prob & label).sum().float()
    TN = ((~prob) & (~label)).sum().float()
    FP = (prob & (~label)).sum().float()
    FN = ((~prob) & label).sum().float()

    precision = torch.mean(TP / (TP + FP + 1e-12))
    recall = torch.mean(TP / (TP + FN + 1e-12))
    F2 = (1 + beta**2) * precision * recall / (beta**2 * precision + recall + 1e-12)
    return F2.mean(0)

In [33]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def fit_one_cycle(epochs, max_lr, model, freeze, train_loader, val_loader, 
                  weight_decay=0, grad_clip=None, opt_func=optim.Adam):
    model.train()
    torch.cuda.empty_cache()
    history = []
    
    # Set up cutom optimizer with weight decay
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    # Set up one-cycle learning rate scheduler
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, 
                                                steps_per_epoch=len(train_loader))
    # Freeze or Unfreeze
    set_parameter_requires_grad(model, freeze)
    for epoch in range(epochs):
        # Training Phase 
        
        train_losses = []
        lrs = []
        for batch in tqdm(train_loader):
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            
            # Gradient clipping
            if grad_clip: 
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
            
            optimizer.step()
            optimizer.zero_grad()
            
            # Record & update learning rate
            lrs.append(get_lr(optimizer))
            sched.step()
        
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [34]:
# Train all layers
model = MyModel(model_name)
model = model.to(device)
epochs = 10
max_lr = 3e-3
freeze = False
history = fit_one_cycle(epochs, 
                        max_lr,
                        model,
                        freeze, 
                        train_dl, 
                        val_dl)


Epoch [0], last_lr: 0.0008, train_loss: 1.0046, val_loss: 0.9190, val_score: 0.8075



Epoch [1], last_lr: 0.0023, train_loss: 1.2688, val_loss: 1.0065, val_score: 0.7717



Epoch [2], last_lr: 0.0030, train_loss: 1.0106, val_loss: 1.0170, val_score: 0.7711



Epoch [3], last_lr: 0.0029, train_loss: 0.9355, val_loss: 0.8500, val_score: 0.8051



Epoch [4], last_lr: 0.0024, train_loss: 0.8464, val_loss: 0.8214, val_score: 0.8083



Epoch [5], last_lr: 0.0018, train_loss: 0.7594, val_loss: 0.7725, val_score: 0.8218



Epoch [6], last_lr: 0.0012, train_loss: 0.6640, val_loss: 0.6998, val_score: 0.8318



Epoch [7], last_lr: 0.0006, train_loss: 0.5562, val_loss: 0.5324, val_score: 0.8761



Epoch [8], last_lr: 0.0001, train_loss: 0.4706, val_loss: 0.4508, val_score: 0.9060



Epoch [9], last_lr: 0.0000, train_loss: 0.4313, val_loss: 0.4295, val_score: 0.9111


In [35]:
# Train all layers
#history += fit_one_cycle(epochs,
#                         max_lr/10,
#                         model,
#                         freeze,
#                         train_dl,
#                         val_dl)